In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
@author:    gkweston
github:     https://github.com/gkweston

This script takes merged data files and marks stress moments 60 seconds before and after user
flagged instances. Then data is windowed by 120 second intervals, dropping any window w/ at
least 20 missing heart rate values. As windows are processed, a file is compiled for each
participant with applicably extracted heartrate features.
"""

import glob as gb
import pandas as pd
import numpy as np

In [2]:
masterDir = r"/Users/gkweston/git/ptsd-continuous-monitoring/data/"
inputDir = masterDir + r"prep_output/pre_window/"
outputDir = masterDir + r"windowed/"
featureDir = masterDir + r"/features/"
monitorDir = r"/Users/gkweston/Desktop/workingFiles/acelab/proj2/data_monitor/"
fileList = gb.glob(inputDir + "*.csv")

In [3]:
# Creating empty dataframe to extract features
fCols=['window_num', 'participant', 'mean_hr', 
       'min_hr', 'max_hr', 'range_hr', 'smoment']
feat_df = pd.DataFrame(columns=fCols)

# Cleaning up dataframe and updating stressmoment 60 s before and 60 after recorded time
for file in fileList:
    participant = file[87:89]
    open_file = pd.read_csv(file, low_memory=False)
    prim_df = pd.DataFrame(open_file)
    prim_df.drop('V1', axis=1, inplace=True)
    prim_df.drop('Unnamed: 0', axis=1, inplace=True)
    prim_df['window'] = np.nan
    momentIndex = prim_df[prim_df['smoment']==1].index
    for moment in momentIndex:
        prim_df['smoment'][moment-60 : moment+60]=1
    
# Creating temporary dataframe, if leq 20 missing hr in temp append to final dataframe
# and add window num
    fin_df = pd.DataFrame()
    nanThresh = 20
    winNum = 1
    
    for i in range(0, prim_df.shape[0], 120):
        temp_df = prim_df[i : i + 120]
        hrNanCount = temp_df['hr'].isna().sum()
        smCount = temp_df.smoment.count()

        if hrNanCount < nanThresh:
#     Updating stress moments in each window
            if smCount > 0:
                temp_df['smoment'] = 1
                stress = 1
            else:
                stress = 0
                
            temp_df['window'] = winNum
#     Extracting features by window specific DataFrame
            meanHr = temp_df['hr'].mean(skipna=True)
            minHr = temp_df['hr'].min(skipna=True)
            maxHr = temp_df['hr'].max(skipna=True)
            rngeHr = maxHr - minHr
            tFeatData = [[winNum, participant, meanHr, minHr, maxHr, rngeHr, stress]]
            tFeat_df = pd.DataFrame(data=tFeatData, columns=fCols)
            
            feat_df = feat_df.append(tFeat_df)
            fin_df = fin_df.append(temp_df)
            winNum += 1
            
# Output final data sets
    fin_df.to_csv(outputDir + file[75:])
    fin_df.to_csv(monitorDir + file[75:])

feat_df.to_csv(featureDir + "feature.csv")
    
print(f"\nAll processes complete succesfuly.")

/Users/gkweston/PyEnvs/ptsdcm/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/gkweston/PyEnvs/ptsdcm/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/gkweston/PyEnvs/ptsdcm/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py


All processes complete succesfuly.
